# Homework 2 - Wikipedia Web Traffic Time Series

У вас есть данные по посещению 1000 страниц  Википедии из разных стран и разных девайсов ( \*  * данные взяты из [Kaggle соревнования](https://www.kaggle.com/c/web-traffic-time-series-forecasting)* )

*wikipedia_train* и *wikipedia_test* - содержат данные о трафике. Это файлы csv, где каждая строка соответствует определенной статье, и каждый столбец соответствует конкретной дате. В некоторых записях отсутствуют данные. Названия страниц содержат проект Википедии (например, en.wikipedia.org), тип доступа (например, desktop) и тип агента (например, spider). Другими словами, каждое имя статьи имеет следующий формат: «name_project_access_agent» (например, «AKB48_zh.wikipedia.org_all-access_spider»).

Вам нужно ответить на [вопросы](https://docs.google.com/forms/d/e/1FAIpQLScQYrgg50C1hJeeNQflumKqJ4rWihDj9Q9HWMy-ls7qt3iAJA/viewform?usp=sf_link) и попробовать сделать самую простую модель которая сможет предсказывать будущие посещения. 

Вот примеры временных рядов посещаемости страниц Википедии (*синие* - обучающая выборка, *зеленые* - предсказания модели победителя соревнования на Kaggle, *оранжевые* - реальные значения):
![Wikipedia Web Traffic Time Series](https://image.ibb.co/cUpEJa/predictions.png)

In [84]:
import re
import pandas as pd
import numpy as np

In [85]:
train = pd.read_csv("./data/wikipedia_train.csv")
test = pd.read_csv("./data/wikipedia_test.csv")

## Data Analysis

In [86]:
def get_language(page):
    res = re.search('[a-z][a-z].wikipedia.org',page)
    if res:
        return res.group(0)[0:2]
    return 'na'

In [87]:
train['country'] = train['Page'].apply(get_language)
rus_len = len(train[train['country'] == 'ru'])
print(f'{rus_len} страниц из русской Википедии в датасете')


102 страниц из русской Википедии в датасете


In [88]:
pd.set_option('display.max_rows', 102)
train_rus = train[train['country'] == 'ru']
train_rus.loc[:, 'sum_views'] = train_rus.iloc[:, 1:-1].sum(axis=1)
sorted_rus = train_rus.sort_values(by=['sum_views'],ascending=False)
print(f"Most popular article {sorted_rus.Page.iloc[0]}")

Most popular article Facebook_ru.wikipedia.org_desktop_all-agents


/home/i.kozlov/venv/mlcourse/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


## Forecasting

Нужно преобразовать `train` данные в следующий формат:

In [89]:
train.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-08-23,2016-08-24,2016-08-25,2016-08-26,2016-08-27,2016-08-28,2016-08-29,2016-08-30,2016-08-31,country
0,15._November_de.wikipedia.org_desktop_all-agents,32.0,26.0,22.0,22.0,29.0,49.0,20.0,27.0,19.0,...,23.0,31.0,25.0,27.0,23.0,17.0,26.0,23.0,37.0,de
1,2012_(film)_fr.wikipedia.org_all-access_spider,2.0,3.0,5.0,3.0,5.0,3.0,7.0,8.0,7.0,...,5.0,6.0,5.0,4.0,11.0,2.0,0.0,7.0,5.0,fr
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,1.0,3.0,2.0,2.0,1.0,10.0,2.0,1.0,4.0,...,3.0,2.0,3.0,2.0,4.0,2.0,0.0,5.0,4.0,ja
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,3.0,3.0,3.0,8.0,12.0,12.0,8.0,12.0,23.0,...,14.0,26.0,5.0,29.0,23.0,17.0,16.0,12.0,14.0,en
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,24.0,40.0,23.0,49.0,88.0,25.0,31.0,76.0,51.0,...,162.0,208.0,179.0,108.0,99.0,49.0,80.0,113.0,173.0,en


In [90]:
new_train = pd.melt(train, id_vars=['Page'], value_vars=train.columns[1:-1]).sort_values(by=['variable', 'Page'])

In [91]:
new_train

,Page,variable,value
0,15._November_de.wikipedia.org_desktop_all-agents,2015-07-01,32.0
1,2012_(film)_fr.wikipedia.org_all-access_spider,2015-07-01,2.0
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,2015-07-01,1.0
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,2015-07-01,3.0
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,2015-07-01,24.0
...,...,...,...
401887,黃義雄_zh.wikipedia.org_all-access_all-agents,2016-08-31,20.0
401888,黄渤_zh.wikipedia.org_all-access_all-agents,2016-08-31,414.0
401889,黑嘉嘉_zh.wikipedia.org_all-access_all-agents,2016-08-31,360.0
401890,黒木瞳_ja.wikipedia.org_all-access_spider,2016-08-31,304.0


Таким образом у вас каждая сточка содержит набор фич (`Page`, `date`) и целевую переменную (`Visits`). Преобразовать данные в такой формат поможет функция `pd.melt()` (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html)

Оценивать качество предсказаний мы будем с помощью [SMAPE](https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error) :

In [7]:
def pandas_smape(df):
    df.fillna(0, inplace=True)
    df["SMAPE"] = 200 * np.abs(df["Visits"] - df["pred_Visits"]) / (df["Visits"] + df["pred_Visits"])
    df["SMAPE"].fillna(0, inplace=True)
    return np.mean(df["SMAPE"])

Что б оценить прогноз нужно представить `test` данные в следующем формате:

In [9]:
prediction.head()

NameError: name 'prediction' is not defined

И затем вызвать функцию для подсчета метрики:

In [ ]:
pandas_smape(prediction)

### Last day baseline

Нужно сделать прогноз на основе посещений в последний известный нам день из train (продублировать значение для каждого дня в test)

### Median baseline

Нужно сделать прогноз на основе медианы за последние **30** дней из `train`. 

А затем улучшить предсказания используя информацию выходной это или нет (воспользуйтесь функцией [dayofweek](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DatetimeIndex.dayofweek.html) ) и разные окна для подсчета медианы (7 дней, 60 дней и тд)

Вам поможет функция `pd.groupby()` (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)